In [2]:
import gensim.downloader

In [3]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

def find_synonym(word):
    return glove_vectors.most_similar(word)[0][0]

[==================================================] 100.0% 376.1/376.1MB downloaded


In [4]:
from datasets import load_dataset
import markovify

dataset = load_dataset("biglam/gutenberg-poetry-corpus")
dataset['train'] = dataset['train'][:12000]

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
text_model = markovify.Text(dataset['train']['line'])
for i in range(5):
    print(text_model.make_sentence())

But least his heart within him Hell
His journies end and our Joy upraise
Here for his fault amerc't
After the Tempest: Such applause was heard declar'd the Almightie's will;
Hissed among the pinetrees,


In [6]:
lines = []
for i in range(5):
    lines.append(text_model.make_sentence())

for line in lines:
    print(line)

Oh the long reeds by the South-wind,
An Atom, with the tongue
Spake entreating, said in the husking
That the time may come when men
On him who sent the fiery serpents,


In [7]:
from textblob import TextBlob
import pandas as pd

def sentiment_analysis(poetry):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity

    def getPolarity(text):
        return TextBlob(text).sentiment.polarity

    poetry['TextBlob_Subjectivity'] = poetry['line'].apply(getSubjectivity)
    poetry['TextBlob_Polarity'] = poetry['line'].apply(getPolarity)

    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'

    poetry['TextBlob_Analysis'] = poetry['TextBlob_Polarity'].apply(getAnalysis)
    
    return poetry

# Poem example
poem = {
    'line': lines
}

poetry_df = pd.DataFrame(poem)

result = sentiment_analysis(poetry_df)
print(result)


                                    line  TextBlob_Subjectivity  \
0   Oh the long reeds by the South-wind,                    0.4   
1               An Atom, with the tongue                    0.0   
2  Spake entreating, said in the husking                    0.0   
3        That the time may come when men                    0.0   
4    On him who sent the fiery serpents,                    0.0   

   TextBlob_Polarity TextBlob_Analysis  
0              -0.05          Negative  
1               0.00           Neutral  
2               0.00           Neutral  
3               0.00           Neutral  
4               0.00           Neutral  


In [8]:
def replace_with_sinonims(poem):
    new_poem = []
    for line in poem:
        new_vers = ''
        for word in line.split(' '):
            from numpy.random import randint
            p = randint(1,101)
            if p % 2 == 0:
                try:
                    word = find_synonym(word)
                except:
                    word = word
            new_vers += word + ' '
        new_poem.append(new_vers)

    return new_poem

new_poem = replace_with_sinonims(lines)
for line in new_poem:
    print(line)

Oh the short grasses by the South-wind, 
An Atom, both of tongue 
Spake entreating, said where the hulling 
That of time might coming when women 
On him whom sent of rhetoric serpents, 


In [9]:
from nltk.translate.bleu_score import sentence_bleu

reference = [[word for word in line] for line in dataset['train']['line']]

candidate = [word for line in lines for word in line ]

score = sentence_bleu(reference, candidate)

print(score)

0.8574286908456553
